### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2022 ###

# Road Trip Optimizer #

#### Alex Gilmore (asgilmore@wisc.edu), Nandan Venkatesan (nvenkatesan2@wisc.edu), and Brendan Zimmer (btzimmer@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-Model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-Discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

add from google doc

## 2. Mathematical Model ##

We are utilizing an MCNF framework to solve our primary problem. The source and sink will be identical since we are interested in only round trips and we will be implementing a minimum trip length to prevent the model from cutting the trip short to reduce cost.

let p be a list of national parks and our starting location

let x be a vector representing if an arc is chosen

let e be a vector of entrance fees for each entered park of an arc

let l be a vector of est. lodging fees for each entered park of an arc

let s be a vector of length of stays for each entered park of an arc

let g be a vector of gas prices for each arc

let D be a vector of  arc distances (miles) between each set of parks

let T be a vector of arc times (s) between each set of parks

let M be the maximum days allowed to travel

let m be the minimum days allowed to travel

let b be a vector of "supply/demand" at each park

let c be a vector of the total lodging and entrance fee cost of each park

$c = s * l * e$

$g_{ij} =  \frac{stateGas_i + stateGas_j}{2} * \frac{1}{milesPerGallon} * D_{ij}$

\begin{align*}
\underset{x}{\max} \ & - \underset{(ij) \in p}\Sigma\ c_{ij} x_{ij} - \underset{(ij) \in p}\Sigma\ g_{ij} x_{ij}  & \\
\text{s.t.} \ & \underset{(ij) \in p}\Sigma\ s_{ij} x_{ij} + 86400*\underset{(ij) \in p}\Sigma\ T_{ij} x_{ij} \le M &\\
& - \underset{(ij) \in p}\Sigma\ s_{ij} x_{ij} - 86400* \underset{(ij) \in p}\Sigma\ T_{ij} x_{ij} \le - m &\\
& \underset{j \in p}\Sigma\ x_{kj} - \underset{i \in p}\Sigma\ x_{ik} \le b_k,\ \forall\ k \in p &\\
& - \underset{j \in p}\Sigma\ x_{kj} + \underset{i \in p}\Sigma\ x_{ik} \le - b_k,\ \forall\ k \in p &\\
\end{align*}



In [ ]:
#constraints
# travel max
# travel min
# park selection

Variables:

- $x_{ij}$ = $\left\{ \begin{array}{ll}
        1 \ \mbox{if arc $x_{ij}$ is in road trip path} \\
        0 \ \mbox{otherwise} & 
    \end{array} \right\}$

Constraints:
1. The total time traveled must be less than the maximum travel time.


2. The total time traveled must be more than the minimum desired travel time. This prevents the model from decreasing the trip length in an effort to reduce the trip cost. 


3. This is a flow balance constraint to direct our road trip path. Only one arc (road) may enter a node (park) and only one arc may leave. This constraint is split into two halves (the third and fourth constraint listed) to display the mathematical model in standard form.

Objective Function:
- This is minimzing the collective cost of visiting parks + the gas cost of traveling between all parks. 

## 3. Solution ##

## STILL CLEANING DATA FOR USE aug 8

In [20]:
using Clp, JuMP, NamedArrays, CSV, DataFrames

In [127]:
df = DataFrame(CSV.File("park_data.csv"))

push!(df, ("UW_Madison", "", 0, 0, 0, 3.5))

pTemp = df[!, "park name"]
p = Array{Symbol}(undef, length(pTemp))

# format parks list to symbols
for i in [1:1:length(pTemp);]
    pTemp[i] = replace(pTemp[i], " " => "_", "–" => "_", "." => "")
    p[i] = Symbol(pTemp[i])
end 

e = NamedArray(df[!, "entrance fee"], (p))

l = NamedArray(df[!, "lodging"], (p))

s = NamedArray(df[!, "est. length of stay"], (p))

stateGas = NamedArray(df[!, "gas price"], (p)) 

c = s .* l .* e

;

58-element Named Vector{Float64}
A                             │ 
──────────────────────────────┼────────
:Acadia                       │ 30000.0
:Arches                       │  3540.0
:Badlands                     │  5100.0
:Big_Bend                     │ 17400.0
:Biscayne                     │     0.0
:Black_Canyon_of_the_Gunnison │  2400.0
:Bryce_Canyon                 │  6300.0
:Canyonlands                  │  3660.0
:Capitol_Reef                 │  4360.0
:Carlsbad_Caverns             │  4560.0
:Congaree                     │     0.0
⋮                                     ⋮
:Sequoia                      │  6790.0
:Shenandoah                   │  3900.0
:Theodore_Roosevelt           │  8100.0
:Voyageurs                    │     0.0
:White_Sands                  │  1800.0
:Wind_Cave                    │     0.0
:Wrangell_St_Elias            │     0.0
:Yellowstone                  │ 27650.0
:Yosemite                     │ 86100.0
:Zion                         │  6720.0
:UW_Madison   

In [54]:
dist = DataFrame(CSV.File("arc_data_dist.csv"))

distO = DataFrame(CSV.File("origin_data_dist.csv"))

time = DataFrame(CSV.File("arc_data_time.csv"))

timeO = DataFrame(CSV.File("origin_data_time.csv"))

dist.UW_Madison = distO[!, 2]
time.UW_Madison = timeO[!, 2]

;

In [131]:
dist.col1 = p[1:57]

50-element Vector{String}:
 "Column1"
 "Acadia National Park"
 "Arches National Park"
 "Badlands National Park"
 "Big Bend National Park"
 "Biscayne National Park"
 "Black Canyon of the Gunnison National Park"
 "Bryce Canyon National Park"
 "Canyonlands National Park"
 "Capitol Reef National Park"
 "Carlsbad Caverns National Park"
 "Congaree National Park"
 "Crater Lake National Park"
 ⋮
 "Rocky Mountain National Park"
 "Saguaro National Park"
 "Sequoia National Park"
 "Shenandoah National Park"
 "Theodore Roosevelt National Park"
 "Voyageurs National Park"
 "White Sands National Park"
 "Wind Cave National Park"
 "Yellowstone National Park"
 "Yosemite National Park"
 "Zion National Park"
 "UW_Madison"